To apply the artificial immune pattern recognition to perform a task of structure damage 
Classification. 

In [1]:
import numpy as np

def create_antibody(size):
  return np.random.rand(size) 

def euclidean_distance(a1, a2):
  return np.linalg.norm(a1 - a2) 

healthy_data = np.array([[1.0, 2.0, 3.0], [1.1, 1.9, 3.2]])
num_antibodies = 10
antibody_population = [create_antibody(healthy_data.shape[1]) for i in range(num_antibodies)]
# Simulate sensor data with potential damage (replace with actual data)
damaged_data = np.array([[1.2, 1.7, 2.8], [1.4, 1.5, 3.5]])

# Affinity (closeness)
def affinity(antibody, datapoint):
  distance = euclidean_distance(antibody, datapoint)
  return 1 / (1 + distance)

for i in range(2):
  healthy_affinities = [affinity(ab, datapoint) for ab in antibody_population for datapoint in healthy_data]
  # Select top 'n' antibodies based on affinity (healthy data)
  top_antibodies = sorted(zip(antibody_population, healthy_affinities), key=lambda x: x[1], reverse=True)[:5]
  # Clone and introduce random mutations (simplified)
  new_population = []
  for ab, i in top_antibodies:
    new_population.append(ab + np.random.randn(ab.shape[0]) * 0.1)  # Introduce small mutation
    antibody_population = new_population
# Update antibody population
antibody_population.extend(new_population)
# Check affinity for damaged data
damaged_affinities = [affinity(ab, damaged_data[0]) for ab in antibody_population]
# Identify potential damage based on high affinity for damaged data
potential_damage_index = damaged_affinities.index(max(damaged_affinities))
print(len(antibody_population))
print("Potential damage detected with antibody:", antibody_population[potential_damage_index])

10
Potential damage detected with antibody: [0.59411875 0.62941863 0.99504881]


In [1]:
import numpy as np

# Generate "self" (normal) data
np.random.seed(42)
self_data = np.random.normal(loc=0.0, scale=1.0, size=(50, 3))  # 50 samples, 3 features

# Generate "non-self" (damaged) data
nonself_data = np.random.normal(loc=3.0, scale=1.0, size=(10, 3))  # Damaged pattern

# Generate detectors (not similar to any self data)
detectors = []
while len(detectors) < 10:
    candidate = np.random.uniform(-5, 5, 3)
    if all(np.linalg.norm(candidate - s) > 1.5 for s in self_data):
        detectors.append(candidate)

# Test samples (some normal, some damaged)
test_samples = np.array([
    [0.2, -0.1, 0.3],  # Normal
    [2.5, 3.1, 2.9],   # Damaged
    [0.0, 0.1, -0.2],  # Normal
    [3.2, 2.9, 3.3]    # Damaged
])

# Classification
def classify(sample, detectors, threshold=1.5):
    for d in detectors:
        if np.linalg.norm(sample - d) < threshold:
            return "Damaged"
    return "Undamaged"

# Output results
for i, s in enumerate(test_samples):
    result = classify(s, detectors)
    print(f"Sample {i+1}: {s} → {result}")


Sample 1: [ 0.2 -0.1  0.3] → Undamaged
Sample 2: [2.5 3.1 2.9] → Undamaged
Sample 3: [ 0.   0.1 -0.2] → Undamaged
Sample 4: [3.2 2.9 3.3] → Undamaged


In [1]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# Step 1: Create synthetic 2-class data (Healthy = 0, Damaged = 1)
X, y = make_classification(n_samples=200, n_features=4, n_classes=2, random_state=1)

# Step 2: Split & scale
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Step 3: Use damaged samples to generate detectors
detectors = X_train[y_train == 1][:20]  # Take 20 detectors from class 1

# Step 4: Simple AIS-style classification
def classify(sample, detectors, radius=1.2):
    for d in detectors:
        if np.linalg.norm(sample - d) < radius:
            return 1  # Detected as damaged
    return 0  # Otherwise healthy

# Step 5: Predict and evaluate
preds = [classify(x, detectors) for x in X_test]
accuracy = np.mean(preds == y_test)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.75
